In [30]:
import pandas as pd
from collections import Counter
from utils import parse_cdhit, replace_val
import itertools


In [22]:
data = pd.read_table("data/ENA_ML_input_ORFs")


In [23]:
data.head(2)

,Feature_id,contig_id,position_start,position_end,ORF_location_on_contig,annotation_val
0,ENA_AY095314_AY095314.2_3724_3921_10,AY095314.2,3724,3921,10,Unknown
1,ENA_JX889246_JX889246.1_45181_44900_68,JX889246.1,45181,44900,68,Uncharacterized protein


In [24]:
seq_to_clust = parse_cdhit("data/ENA.40.clstr")

# what does it look like
list(seq_to_clust.items())[0:2]

[('ENA_KP211958_KP211958.1_65082_87137_88', '0'),
 ('ENA_AY940168_AY940168.2_57144_79082_90', '1')]

In [25]:
## Add a column to show the cluster to which each feature_id belongs
data["feature_number"] = replace_val(data.Feature_id, seq_to_clust)
data.feature_number = data.feature_number.astype(int)
data.head(2)



,Feature_id,contig_id,position_start,position_end,ORF_location_on_contig,annotation_val,feature_number
0,ENA_AY095314_AY095314.2_3724_3921_10,AY095314.2,3724,3921,10,Unknown,57065
1,ENA_JX889246_JX889246.1_45181_44900_68,JX889246.1,45181,44900,68,Uncharacterized protein,43194


In [28]:
# Add a contig sequential number in addition to contig id.
contig_id_to_int = {y:x for x,y in enumerate(data.contig_id.tolist())}
#### data.replace({'contig_id': contig_id_to_int})
data["contig_number"] = replace_feature(data.contig_id, contig_id_to_int)
data.head(2)


,Feature_id,contig_id,position_start,position_end,ORF_location_on_contig,annotation_val,feature_number,contig_number
0,ENA_AY095314_AY095314.2_3724_3921_10,AY095314.2,3724,3921,10,Unknown,57065,237296
1,ENA_JX889246_JX889246.1_45181_44900_68,JX889246.1,45181,44900,68,Uncharacterized protein,43194,234989


* Count the occurrences of each pair of features that occur on at least 1 contig

* We generate generate all the possible pairs (tuples) of features 


In [33]:
# Generate all the subset of size 2 for each contig
subsets_size_2 = data.groupby("contig_number").apply(lambda x: list(itertools.combinations(x["feature_number"].values, 2)))
subsets_size_2.head(2)


contig_number
82638     [(3929, 6051), (3929, 21058), (3929, 8867), (3...
106079    [(18722, 6292), (18722, 30149), (18722, 3490),...
dtype: object

In [34]:
subsets_size_2.iloc[1]

[(18722, 6292),
 (18722, 30149),
 (18722, 3490),
 (6292, 30149),
 (6292, 3490),
 (30149, 3490)]

* We make sure the tuple is sorted so it's easy to compute frequencies

In [40]:
co_occurences_2 = []
for item in list(itertools.chain(*subsets.values)):
    if item[0] > item[1]:
        co_occurences_2.append((item[1], item[0]))
    else:
        co_occurences_2.append((item[1], item[0]))  
co_occurences_2[:10]    

[(6051, 3929),
 (21058, 3929),
 (8867, 3929),
 (57763, 3929),
 (25595, 3929),
 (21058, 6051),
 (8867, 6051),
 (57763, 6051),
 (25595, 6051),
 (8867, 21058),
 (57763, 21058),
 (25595, 21058),
 (57763, 8867),
 (25595, 8867),
 (25595, 57763),
 (6292, 18722),
 (30149, 18722),
 (3490, 18722),
 (30149, 6292),
 (3490, 6292),
 (3490, 30149),
 (21058, 6051),
 (8867, 6051),
 (57763, 6051),
 (25595, 6051),
 (3929, 6051),
 (8867, 21058),
 (57763, 21058),
 (25595, 21058),
 (3929, 21058),
 (57763, 8867),
 (25595, 8867),
 (3929, 8867),
 (25595, 57763),
 (3929, 57763),
 (3929, 25595),
 (30149, 6292),
 (26181, 6292),
 (26181, 30149),
 (6051, 8867),
 (3929, 8867),
 (44650, 8867),
 (25595, 8867),
 (21058, 8867),
 (3929, 6051),
 (44650, 6051),
 (25595, 6051),
 (21058, 6051),
 (44650, 3929),
 (25595, 3929),
 (21058, 3929),
 (25595, 44650),
 (21058, 44650),
 (21058, 25595),
 (30940, 6816),
 (4069, 6816),
 (4069, 30940),
 (5683, 31590),
 (3986, 31590),
 (3986, 5683),
 (30940, 6816),
 (4069, 6816),
 (4069, 30

* Count the number of occurrences of each pair of features

In [43]:
# co_occurences__2_counts = Counter(co_occurences)
pd.DataFrame
list(co_occurences__2_counts.items())[:3]

[((3929, 6051), 62), ((3929, 21058), 64), ((3929, 8867), 66)]